<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/LSTM_PRVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Price Rationality Verification System

* 가격 합리성 검증 시스템

### Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/DeepLearning/Project/PRVS
!ls -al

Mounted at /content/gdrive
/content/gdrive/MyDrive/DeepLearning/Project/PRVS
total 14
drwx------ 2 root root  4096 Jan 23 02:35 Dataset
-rw------- 1 root root 10180 Feb  7 11:24 LSTM-PRVS.ipynb


### Install

In [2]:
!pip install beautifulsoup4

### Import

In [3]:
# Crawling
import requests
from bs4 import BeautifulSoup

# Modeling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import metrics, losses
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

# ETC
from collections import Counter

### Global Variable

In [4]:
# LED가로등기구, 39111603
CLASSIFI_NUM = 39111603
IDENTITY_NUM = 23435411

### Main

In [5]:
# 나라장터 가격 크롤링 불가, 배추김치(포기김치, 송이식품) 1kg 가격
price_go = 365000

# 네이버 가격 크롤링
url_naver = 'https://search.shopping.naver.com/search/all?query=LED%EA%B0%80%EB%A1%9C%EB%93%B1%EA%B8%B0%EA%B5%AC&cat_id=&frm=NVSHATC'
response = requests.get(url_naver)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    ul = soup.select_one('#__next > div > div.style_container__1YjHN > div')

    prices = ul.select('div > div > ul > div > div > li > div > div > div > strong > span > span')
    for idx, price in enumerate(prices) : 
        prices[idx] = int(price.get_text().split(',')[0] + price.get_text().split(',')[1][:-1])

    prices = np.array(prices)
    print(prices)
    mean = np.mean(prices)
    std = np.std(prices)
    cov = std / mean * 100
    print('평균 = ', mean)
    print('표준편차 = ', std)
    print('변동 계수 = ', cov)

    print('\n########## 최대값 제거 ##########')
    prices = np.delete(prices, np.where(prices == max(prices)))
    print(prices)
    mean = np.mean(prices)
    std = np.std(prices)
    cov = std / mean * 100
    print('평균 = ', mean)
    print('표준편차 = ', std)
    print('변동 계수 = ', cov)

    """
    print('\n########## 최소값 제거 ##########')
    prices = np.delete(prices, np.where(prices == min(prices)))
    print(prices)
    mean = np.mean(prices)
    std = np.std(prices)
    cov = std / mean * 100
    print('평균 = ', mean)
    print('표준편차 = ', std)
    print('변동 계수 = ', cov)
    """

    print('\n########## 결과값 ##########')
    mean = int(round(np.mean(prices), -1))
    print('나라장터 가격 = ', price_go)
    print('네이버쇼핑 평균가 = ', mean)
    print('가격 차이 = ', abs(price_go - mean))
else : 
    print(response.status_code)

[226000 336000 188000 460000 156000]
평균 =  273200.0
표준편차 =  111409.87388916657
변동 계수 =  40.77960244845043

########## 최대값 제거 ##########
[226000 336000 188000 156000]
평균 =  226500.0
표준편차 =  67902.5036357276
변동 계수 =  29.979030302749493

########## 결과값 ##########
나라장터 가격 =  365000
네이버쇼핑 평균가 =  226500
가격 차이 =  138500


In [10]:
df = pd.read_csv('./Dataset/LED_streetlight_23435411_202201.csv')

itmClsNmb = df['물품분류번호']
itmIdnNmb = df['물품식별번호']
untPrc = df['단가']
print(itmClsNmb)
print(itmIdnNmb)
print(untPrc)

incDcr = df['증감수량']
incDcrAmn = df['증감금액']
incDcrDlvRqrQnt = df['증감납품요구수량']
incDcrDlvRqsAmn = df['증감납품요구금액']
print(incDcr)
print(incDcrAmn)
print(incDcrDlvRqrQnt)
print(incDcrDlvRqsAmn)

excPrd = df['우수제품여부']
cmpAmnSms = df['중소기업자간경쟁제품여부']
drcPrc = df['공사용자재직접구매대상여부']
mltSplCnt = df['다수공급자계약여부']
print(excPrd)
print(cmpAmnSms)
print(drcPrc)
print(mltSplCnt)

ttlQnt = df['수량합계']
ttlAmn = df['금액합계']
avrUntPrc = df['평균단가']
print(ttlQnt)
print(ttlAmn)
print(avrUntPrc)

0    39111603
1    39111603
2    39111603
3    39111603
Name: 물품분류번호, dtype: int64
0    23435411
1    23435411
2    23435411
3    23435411
Name: 물품식별번호, dtype: int64
0    365000
1    365000
2    365000
3    365000
Name: 단가, dtype: int64
0     1
1    56
2    35
3     6
Name: 증감수량, dtype: int64
0      365000
1    20440000
2    12775000
3     2190000
Name: 증감금액, dtype: int64
0     1
1    56
2    98
3     6
Name: 증감납품요구수량, dtype: int64
0      365000
1    20440000
2    39345000
3     2190000
Name: 증감납품요구금액, dtype: int64
0    Y
1    Y
2    Y
3    Y
Name: 우수제품여부, dtype: object
0    Y
1    Y
2    Y
3    Y
Name: 중소기업자간경쟁제품여부, dtype: object
0    Y
1    Y
2    Y
3    Y
Name: 공사용자재직접구매대상여부, dtype: object
0    N
1    N
2    N
3    N
Name: 다수공급자계약여부, dtype: object
0    98
1    98
2    98
3    98
Name: 수량합계, dtype: int64
0    35770000
1    35770000
2    35770000
3    35770000
Name: 금액합계, dtype: int64
0    365000
1    365000
2    365000
3    365000
Name: 평균단가, dtype: int64
